In [1]:
# https://qiita.com/matsulib/items/898873b73d584c7dcb8b
import numpy as np, pandas as pd


In [2]:
skills = ["Controller", "Promoter", "Supporter", "Analyzer"]
members = ["P", "Q", "R", "S", "T", "U"]
teams = ["A", "B", "C"]
scores = pd.DataFrame([
    [6,0,1,3],
    [2,-1,3,5],
    [2,4,0,0],
    [3,4,5,0],
    [0,2,1,4],
    [2,3,-1,1]
], columns=skills, index=members)

num_teams = len(teams)
num_members = scores.shape[0]
print(f"makes {num_teams} teams with {num_members} members")

makes 3 teams with 6 members


In [3]:
scores

,Controller,Promoter,Supporter,Analyzer
P,6,0,1,3
Q,2,-1,3,5
R,2,4,0,0
S,3,4,5,0
T,0,2,1,4
U,2,3,-1,1


In [4]:
from pulp import LpProblem, lpSum, value, lpDot
from ortoolpy import addvars, addbinvars, addvar

In [5]:

m = LpProblem()  # 数理モデル
x = pd.DataFrame(addbinvars(len(members), len(teams)), index=members, columns=teams)  # 割当
y = pd.DataFrame(addvars(len(teams), len(skills)), index=teams, columns=skills)  # チーム内の平均偏差
mu = pd.DataFrame(addvars(len(teams)), index=teams)   # チーム内の平均
z = pd.DataFrame(addvars(len(teams)), index=teams)  # チームごとの平均偏差
nu = addvar()  # 全チームの平均

In [7]:

m.setObjective(lpSum([lpSum(y.loc[j]) + 1.5*z.loc[j] for j in teams]))  # 目的関数

m.addConstraint(lpSum(np.dot(scores.sum(1), x)) / len(teams) == nu)
for j in teams:
    m.addConstraint(lpDot(scores.sum(1), x[j]) - nu <= z.loc[j])
    m.addConstraint(lpDot(scores.sum(1), x[j]) - nu >= -z.loc[j])
    m.addConstraint(lpSum(np.dot(x[j], scores)) / len(skills) == mu.loc[j])
    for k in skills:
        m.addConstraint(lpDot(scores[k], x[j]) - mu.loc[j] <= y.loc[j,k])
        m.addConstraint(lpDot(scores[k], x[j]) - mu.loc[j] >= -y.loc[j,k])
for i in members:
    m.addConstraint(lpSum(x.loc[i]) == 1)  # どこかのチームに所属



In [8]:
m.solve()  # 求解


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/toshiya/Library/Caches/pypoetry/virtualenvs/combinatorial-optimization-practice-O_WzHL3X-py3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_x/8s9ykbp91d39v67y3yj2j_fc0000gn/T/6e5d6591fd3648b6a92af3beabe409a2-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/_x/8s9ykbp91d39v67y3yj2j_fc0000gn/T/6e5d6591fd3648b6a92af3beabe409a2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 85 COLUMNS
At line 673 RHS
At line 754 BOUNDS
At line 773 ENDATA
Problem MODEL has 80 rows, 37 columns and 536 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7 - 0.00 seconds
Cgl0004I processed model has 40 rows, 37 columns (18 integer (18 of which binary)) and 268 elements
Cbc0038I Initial state - 13 integers unsatisfied sum - 3.94903
Cbc0038I Pass   1: suminf.

1

In [9]:

if m.status == 1:
    for i, xi in enumerate(np.vectorize(value)(x).tolist()):
        print((members[i], teams[xi.index(1)]))
else:
    print('最適解を得られなかった。')

('P', 'B')
('Q', 'A')
('R', 'A')
('S', 'C')
('T', 'B')
('U', 'C')
